In [9]:
import httpx
from parsel import Selector

response = httpx.get(
    "https://www.glassdoor.com/Overview/Working-at-eBay-EI_IE7853.11,15.htm"
)
selector = Selector(response.text)
# find description in the HTML:
print(selector.css('[data-test="employerDescription"]::text').get())
# will print:
# eBay is where the world goes to shop, sell, and give. Every day, our professionals connect millions of buyers and sellers around the globe, empowering people and creating opportunity. We're on a mission to build a better, more connected form of commerce that benefits individuals

None


In [10]:
function addGlobalStyle(css) {
    var head, style;
    head = document.getElementsByTagName('head')[0];
    if (!head) { return; }
    style = document.createElement('style');
    style.type = 'text/css';
    style.innerHTML = css;
    head.appendChild(style);
}

addGlobalStyle("#HardsellOverlay {display:none !important;}");
addGlobalStyle("body {overflow:auto !important; position: initial !important}");

window.addEventListener("scroll", event => event.stopPropagation(), true);
window.addEventListener("mousemove", event => event.stopPropagation(), true);

SyntaxError: invalid syntax (108858732.py, line 1)

In [11]:
from parsel import Selector
import httpx

france_location_cookie = {"tldp": "6"}
response = httpx.get(
    "https://www.glassdoor.com/Overview/Working-at-eBay-EI_IE7853.11,15.htm",
    cookies=france_location_cookie,
    follow_redirects=True,
)
selector = Selector(response.text)
# find employee count in the HTML:
print(selector.css('[data-test="employer-size"]::text').get())
# will print:
# Plus de 10 000 employés

None


In [23]:
employer_name = 'Amazon-Milan'
employer_id = 'I_IE6036.0,6_IL.7,12_IM1058'

In [24]:
import asyncio
import json
import math
from collections import defaultdict
from typing import Dict, List

import httpx
from parsel import Selector

session = httpx.AsyncClient(
    timeout=httpx.Timeout(30.0),
    cookies={"tldp": "1"},
    follow_redirects=True,
)


def find_json_objects(text: str, decoder=json.JSONDecoder()):
    """Find JSON objects in text, and generate decoded JSON data and it's ID"""
    pos = 0
    while True:
        match = text.find("{", pos)
        if match == -1:
            break
        try:
            result, index = decoder.raw_decode(text[match:])
            # backtrack to find the key/identifier for this json object:
            key_end = text.rfind('"', 0, match)
            key_start = text.rfind('"', 0, key_end)
            key = text[key_start + 1 : key_end]
            yield key, result
            pos = match + index
        except ValueError:
            pos = match + 1


def extract_apollo_cache(html):
    """Extract apollo graphql cache data from HTML source"""
    selector = Selector(text=html)
    script_with_cache = selector.xpath("//script[contains(.,'window.appCache')]/text()").get()
    cache = defaultdict(list)
    for key, data in find_json_objects(script_with_cache):
        cache[key].append(data)
    return cache


def parse_jobs(html) -> List[Dict]:
    """parse jobs page for job data and total amount of jobs"""
    cache = extract_apollo_cache(html)
    return [v["jobview"] for v in cache["JobListingSearchResult"]]


def parse_job_page_count(html) -> int:
    """parse job page count from pagination details in Glassdoor jobs page"""
    _total_results = Selector(html).css(".paginationFooter::text").get()
    if not _total_results:
        return 1
    _total_results = int(_total_results.split()[-1])
    _total_pages = math.ceil(_total_results / 40)
    return _total_pages


async def scrape_jobs(employer_name: str, employer_id: str):
    """Scrape job listings"""
    # scrape first page of jobs:
    first_page = await session.get(
        url=f"https://www.glassdoor.com/Jobs/{employer_name}-Jobs-E{employer_id}.htm?filter.countryId={session.cookies.get('tldp') or 0}",
    )
    jobs = parse_jobs(first_page.text)
    total_pages = parse_job_page_count(first_page.text)

    print(f"scraped first page of jobs, scraping remaining {total_pages - 1} pages")
    other_pages = [
        session.get(
            url=str(first_page.url).replace(".htm", f"_P{page}.htm"),
        )
        for page in range(2, total_pages + 1)
    ]
    for page in await asyncio.gather(*other_pages):
        jobs.extend(parse_jobs(page.text))
    return jobs


async def main():
    jobs = await scrape_jobs("eBay", "7853")
    print(json.dumps(jobs, indent=2))


asyncio.run(main())


AttributeError: 'NoneType' object has no attribute 'find'

In [20]:
import json
import httpx


def find_companies(query: str):
    """find company Glassdoor ID and name by query. e.g. "ebay" will return "eBay" with ID 7853"""
    result = httpx.get(
        url=f"https://www.glassdoor.com/searchsuggest/typeahead?numSuggestions=8&source=GD_V2&version=NEW&rf=full&fallback=token&input={query}",
    )
    data = json.loads(result.content)
    return data[0]["suggestion"], data[0]["employerId"]

print(find_companies("ebay"))
["eBay", "7853"]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [26]:
pip install enlighten

                                              0.0/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/53.7 kB ? eta -:--:--
     ---------------                          20.5/5

In [27]:
# Import necessary libraries
# standard libraries
import argparse
import json
import os
import sys
from os.path import exists
from datetime import datetime
from time import time
import csv
# 3rd-party libraries
import enlighten
# custom functions
from packages.common import requestAndParse
from packages.page import extract_maximums, extract_listings
from packages.listing import extract_listing


class glassdoor_scraper():

    def __init__(self, configfile, baseurl, targetnum) -> None:

        # load first
        base_url, target_num = self.load_configs(path=configfile)
        # overwrite those that are not none
        if type(baseurl) != type(None):
            base_url = baseurl
            print("Using supplied baseurl")
        if type(targetnum) != type(None):
            target_num = targetnum
            print("Using supplied targetnum")
        print(configfile, baseurl, targetnum)

         # initialises output directory and file
        if not os.path.exists('output'):
            os.makedirs('output')
        now = datetime.now() # current date and time
        output_fileName = "./output/output_" + now.strftime("%d-%m-%Y") + ".csv"
        csv_header = [("companyName", "company_starRating", "company_offeredRole", "company_roleLocation", "listing_jobDesc", "requested_url")]
        self.fileWriter(listOfTuples=csv_header, output_fileName=output_fileName)

        maxJobs, maxPages = extract_maximums(base_url)
        # print("[INFO] Maximum number of jobs in range: {}, number of pages in range: {}".format(maxJobs, maxPages))
        if (target_num >= maxJobs):
            print("[ERROR] Target number larger than maximum number of jobs. Exiting program...\n")
            os._exit(0)

        # initialises enlighten_manager
        enlighten_manager = enlighten.get_manager()
        progress_outer = enlighten_manager.counter(total=target_num, desc="Total progress", unit="listings", color="green", leave=False)

        # initialise variables
        page_index = 1
        total_listingCount = 0

        # initialises prev_url as base_url
        prev_url = base_url

        while total_listingCount <= target_num:
            # clean up buffer
            list_returnedTuple = []

            new_url = self.update_url(prev_url, page_index)
            page_soup,_ = requestAndParse(new_url)
            listings_set, jobCount = extract_listings(page_soup)
            progress_inner = enlighten_manager.counter(total=len(listings_set), desc="Listings scraped from page", unit="listings", color="blue", leave=False)

            print("\n[INFO] Processing page index {}: {}".format(page_index, new_url))
            print("[INFO] Found {} links in page index {}".format(jobCount, page_index))

            for listing_url in listings_set:

                # to implement cache here

                returned_tuple = extract_listing(listing_url)
                list_returnedTuple.append(returned_tuple)
                # print(returned_tuple)
                progress_inner.update()

            progress_inner.close()

            self.fileWriter(listOfTuples=list_returnedTuple, output_fileName=output_fileName)

            # done with page, moving onto next page
            total_listingCount = total_listingCount + jobCount
            print("[INFO] Finished processing page index {}; Total number of jobs processed: {}".format(page_index, total_listingCount))
            page_index = page_index + 1
            prev_url = new_url
            progress_outer.update(jobCount)

        progress_outer.close()    
   
            # loads user defined parameters
    def load_configs(self, path):
        with open(path) as config_file:
            configurations = json.load(config_file)

        base_url = configurations['base_url']
        target_num = int(configurations["target_num"])
        return base_url, target_num


    # appends list of tuples in specified output csv file
    # a tuple is written as a single row in csv file 
    def fileWriter(self, listOfTuples, output_fileName):
        with open(output_fileName,'a', newline='') as out:
            csv_out=csv.writer(out)
            for row_tuple in listOfTuples:
                try:
                    csv_out.writerow(row_tuple)
                    # can also do csv_out.writerows(data) instead of the for loop
                except Exception as e:
                    print("[WARN] In filewriter: {}".format(e))


    # updates url according to the page_index desired
    def update_url(self, prev_url, page_index):
        if page_index == 1:
            prev_substring = ".htm"
            new_substring = "_IP" + str(page_index) + ".htm"
        else:
            prev_substring = "_IP" + str(page_index - 1) + ".htm"
            new_substring = "_IP" + str(page_index) + ".htm"

        new_url = prev_url.replace(prev_substring, new_substring)
        return new_url

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--configfile', help="Specify location of json config file", type=str, required=False, default="config.json")
    parser.add_argument('-b', '--baseurl', help="Base_url to use. Overwrites config file", type=str, required=False, default=None)
    parser.add_argument('-tn', '--targetnum', help="Target number to scrape. Overwrites config file", type=int, required=False, default=None)
    args = vars(parser.parse_args())

    glassdoor_scraper( 
        configfile=args["configfile"],
        baseurl=args["baseurl"],
        targetnum=args["targetnum"]
        )

ModuleNotFoundError: No module named 'packages'